In [1]:
!pip install openai==0.28
!pip install numpy-stl

  Attempting uninstall: openai
    Found existing installation: openai 1.59.6
    Uninstalling openai-1.59.6:
      Successfully uninstalled openai-1.59.6


In [26]:
import requests
import json

# Replace with your Azure details
endpoint = "https://pgazureaideplo6829607801.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview"
api_key = "f8285b2a596f41cb8b3f36ddb0b2c31a"

headers = {
    "Content-Type": "application/json",
    "api-key": api_key
}

def is_valid_3d_object_prompt(user_prompt):
    system_instruction = (
        "You are an intelligent assistant. Your task is to determine whether the given prompt is specifically "
        "intended for creating a 3D object. Valid prompts should describe a 3D object to be created, including "
        "specific attributes such as dimensions, shapes, or material properties. Return 'Valid' if the prompt "
        "is for creating a 3D object, and 'Invalid' otherwise."
    )

    try:
        response = requests.post(
            endpoint,
            headers=headers,
            json={
                "messages": [
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_prompt}
                ],
                "max_tokens": 10
            }
        )
        if response.status_code == 200:
            result = response.json()["choices"][0]["message"]["content"].strip().lower()
            return result == 'valid'
        else:
            print(f"Error! Status Code: {response.status_code}")
            print(response.json())
            return False
    except Exception as e:
        print(f"Error during validation: {e}")
        return False

def extract_features(user_prompt):
    if not is_valid_3d_object_prompt(user_prompt):
        return {"error": "The prompt is not a valid prompt for creating a 3D object."}

    system_instruction = (
        "You are an intelligent assistant. Your task is to extract all the necessary features, "
        "key dimensions, and attributes for creating a 3D object from the given user prompt and "
        "return the results in structured JSON format. Ensure the JSON is well-formed and human-readable."
    )

    try:
        response = requests.post(
            endpoint,
            headers=headers,
            json={
                "messages": [
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_prompt}
                ],
                "max_tokens": 1000
            }
        )
        if response.status_code == 200:
            raw_content = response.json()["choices"][0]["message"]["content"]
            # Remove backticks and extract the JSON part
            clean_content = raw_content.strip("```").strip()
            if clean_content.startswith("json"):
                clean_content = clean_content[4:].strip()
            return json.loads(clean_content)
        else:
            print(f"Error! Status Code: {response.status_code}")
            print("Response Content:", response.text)
            return {"error": "Failed to process the prompt."}
    except json.JSONDecodeError as json_err:
        print(f"JSON decode error: {json_err}")
        print("Response Content:", response.text)
        return {"error": "Response is not in valid JSON format."}
    except Exception as e:
        print(f"An error occurred: {e}")
        return {"error": "An unexpected error occurred."}


# Example usage
if __name__ == "__main__":
    user_prompt = input("Enter a prompt: ")
    features_json = extract_features(user_prompt)

    if "error" in features_json:
        print(features_json["error"])
    else:
        print(json.dumps(features_json, indent=4))


Enter a prompt:  Design a cylindrical water tank with the following specifications: a diameter of 2.5 meters, a height of 4 meters, and a capacity of 20,000 liters. The material should be stainless steel, and the tank should include an outlet valve at the bottom and a ladder attached to the side. Provide details for the base thickness (5 mm) and wall thickness (3 mm). Specify all dimensions and features in the output.


{
    "object": "cylindrical_water_tank",
    "dimensions": {
        "diameter": 2.5,
        "height": 4
    },
    "capacity_liters": 20000,
    "material": "stainless_steel",
    "features": {
        "outlet_valve": {
            "position": "bottom"
        },
        "ladder": {
            "position": "side"
        },
        "base_thickness_mm": 5,
        "wall_thickness_mm": 3
    }
}
